In [1]:
# Additionally we assume you have needleall install on the system and on the path
from Bio import SeqIO
import numpy as np
from pydivsufsort import divsufsort, kasai
import argparse
import sys
from Bio import pairwise2
import subprocess
import re
from pathlib import Path

In [2]:
def abi_to_sequence(abi_file_path):
    '''Convert an ABI (sanger trace file) to a trimmed sequence using Mott's algorithm'''
    record_seq = ""
    for record in SeqIO.parse(abi_file_path, "abi-trim"):
        if record_seq != "":
            raise NameError("Unknown ABI file "  + abi_file_path)
        record_seq = record.seq
        
    return(record_seq)

def comp_base(base):
    if base == "A":
        return "T"
    if base == "C":
        return "G"
    if base == "G":
        return "C"
    if base == "T":
        return "A"
    return "N"

def reverse_comp(seq):
    return "".join([comp_base(b) for b in seq][::-1])

def find_longest_prefix(string1, string2):
    '''
    Use the pydivsufsort library to find the longest common prefix of two strings. We 
    use this later to orient a circular reference to its sanger result
    '''
    max_suffix = 0
    max_index = 0

    joined = str(string1) + "Z" + str(string2)
    strsuffix_array = divsufsort(joined)
    string_lcp_array = kasai(joined, strsuffix_array)

    for i in range(0, len(strsuffix_array) - 1):
        is_ref1 = strsuffix_array[i] > len(ref)
        is_ref2 = strsuffix_array[i + 1] > len(ref)
        if string_lcp_array[i] > max_suffix and is_ref1 != is_ref2:
            max_suffix = string_lcp_array[i]
            max_index = i

    if strsuffix_array[max_index] < strsuffix_array[max_index + 1]:
        return (
            max_suffix,
            max_index,
            strsuffix_array[max_index],
            strsuffix_array[max_index + 1] - (1 + len(string1)),
        )
    return (
        max_suffix,
        max_index,
        strsuffix_array[max_index + 1],
        strsuffix_array[max_index] - (1 + len(string1)),
    )

def read_multi_seq_fasta(fasta):
    aligned_seqs_file = open("alignment.fa")
    sequences = []
    current_name = ""
    current_seq = ""
    
    for line in aligned_seqs_file:
        if ">" in line:
            if current_name != "":
                sequences.append([current_name,current_seq])
            current_seq = ""
            current_name = line.strip("\n").lstrip(">")
        else:
            current_seq = current_seq + line.strip("\n")

    if current_name != "":
        sequences.append([current_name,current_seq])
    
    return(sequences)

def align(seq1,seq2):
    '''
    Use Needle to align two sequences (assumes you have EMBL needleall installed)
    '''
    out_ref = open("test_ref.fa","w")
    out_ref.write(">ref\n")
    out_ref.write(seq1 + "\n")
    out_ref.close()

    out_seq = open("test_seq.fa","w")
    out_seq.write(">seq\n")
    out_seq.write(seq2 + "\n")
    out_seq.close()

    aligment = subprocess.run(["needleall","-asequence","test_ref.fa","-bsequence","test_seq.fa","-outfile","alignment.fa","-aformat3","fasta","-gapopen","10.0","-gapextend","0.5"])
    if aligment.returncode != 0:
        raise NameError("Return code " + str(aligment.returncode))
    aligned_seqs_file = read_multi_seq_fasta("alignment.fa")
    # print(aligned_seqs_file)
    return(aligned_seqs_file)

def slice_by_alignment(ref,sanger):
    '''
    remove end gaps from an aligned sequence 
    '''
    front = re.search(r'[^-]', sanger).start()

    ttt = list(sanger)
    ttt.reverse()

    back = re.search(r'[^-]', "".join(ttt)).start()
    
    new_ref = ref[front:len(ref) - back]
    new_sanger = sanger[front:len(sanger) - back]
    return([new_ref,new_sanger])


def edit_distance(ref,sanger):
    differences = 0
    assert(len(ref) == len(sanger))
    for b1,b2 in zip(ref.upper(),sanger.upper()):
        if b1 == '-' or b2 == '-' or b1 != b2:
            differences += 1
    return([differences,len(ref)])

In [3]:
def rotate_plasmid(new_offset_location,plasmid):
    '''rotate a plasmid to a new offset location'''
    mid_point = 2000 # int(len(plasmid)/2)
    if mid_point < new_offset_location:
        wrap_dist = new_offset_location - mid_point
        return(plasmid[wrap_dist:len(plasmid)] + plasmid[0:wrap_dist])
    else:
        wrap_dist = mid_point - new_offset_location
        return(plasmid[len(plasmid)-wrap_dist:len(plasmid)] + plasmid[0:len(plasmid)-wrap_dist])
    
    
rotate_plasmid(8,"AAAAATTTTTCCCCCGGGGG")

'AAAAATTTTTCCCCCGGGGG'

In [4]:
import warnings
from os.path import exists

warnings.filterwarnings("ignore")

sample_table = open("fex_117_sample_sheet_sanger.tsv")
header = sample_table.readline()
output_file = open("summary_file_2021_10_24.txt","w")
output_file.write("id\tref\tsangerDir\tsangerFile\tsangerErrors\tsangerLength\tplasmidLength\n")
for line in sample_table:
    tokens = line.strip("\n").split("\t")
    
    # #############################################
    # load up the reference file
    id_number = tokens[0].rjust(2, '0')
    
    ref = ""
    if exists("rotated/" + id_number + "_rotated_reference.fasta"):
        reference_input = open("rotated/" + id_number + "_rotated_reference.fasta")

        ref_header = reference_input.readline().strip()
        for line in reference_input:
            ref += line.strip().upper()

        sanger_section = tokens[3]
        if len(sanger_section) > 5:
            sanger_split = sanger_section.split(",")

            for sanger in sanger_split:
                sanger_name = Path(sanger).name
                sanger_sequence = str(abi_to_sequence("FE_Sanger_ab1/" + sanger_name)).upper()

                fwd = find_longest_prefix(ref, sanger_sequence)
                rev_comp = find_longest_prefix(ref, reverse_comp(sanger_sequence))
                ref_new = ""
                assembled_new = ""
                direction = "FWD-" + str(fwd[0])+ "-" + str(fwd[2])
                if fwd[0] > rev_comp[0]:
                    # we want to make this ~ the center of the refernece
                    ref_new = rotate_plasmid(fwd[2],ref)  #ref[fwd[2] : len(ref)] + ref[0 : fwd[2]]
                    assembled_new = sanger_sequence # [fwd[3] : len(assembly)] + assembly[0 : fwd[3]]
                else:
                    direction = "REV-" + str(rev_comp[0]) + "-" + str(fwd[2])
                    ref_new = rotate_plasmid(fwd[2],ref)  # ref[rev_comp[2] : len(ref)] + ref[0 : rev_comp[2]]
                    assembled_new = reverse_comp(sanger_sequence) # (reverse_comp(assembly)[rev_comp[3] : len(assembly)]+ reverse_comp(assembly)[0 : rev_comp[3]])
                aligned_sequences = align(ref_new, assembled_new)
                slices = slice_by_alignment(aligned_sequences[0][1],aligned_sequences[1][1])

                eval_scores = edit_distance(slices[0],slices[1])
                output_file.write(str(id_number) + "\t" + tokens[1] + "\t" + direction + "\t" + str(sanger) + "\t" + str(eval_scores[0]) + "\t" + str(eval_scores[1]) + "\t" + str(len(ref)) + "\n")
    else:
        print("rotated/" + id_number + "_rotated_reference.fasta doesn't exist")
        
output_file.close()

rotated/96_rotated_reference.fasta doesn't exist
